Store sqllite

- blockid
- \# transactions
- largest transaction
- avg transaction amount
- average fee
- time in gmt (convert)

In [ ]:
import blockcypher as bc
import requests
from datetime import datetime, timezone

token = 'e05a8646a0ed4b55b9fadb731877d9e6'
url = "https://api.blockcypher.com/v1/btc/main?token="+token

In [5]:
block_resp

{'name': 'BTC.main',
 'height': 891433,
 'hash': '000000000000000000012389cc51a5057e087221d73fbac77dc53129fcea1edb',
 'time': '2025-04-08T02:05:51.982568202Z',
 'latest_url': 'https://api.blockcypher.com/v1/btc/main/blocks/000000000000000000012389cc51a5057e087221d73fbac77dc53129fcea1edb',
 'previous_hash': '00000000000000000001a8839215fe72cfea1cae3d3e1531b3da0ce18c6b759d',
 'previous_url': 'https://api.blockcypher.com/v1/btc/main/blocks/00000000000000000001a8839215fe72cfea1cae3d3e1531b3da0ce18c6b759d',
 'peer_count': 324,
 'unconfirmed_count': 1996,
 'high_fee_per_kb': 3547,
 'medium_fee_per_kb': 2262,
 'low_fee_per_kb': 1709,
 'last_fork_height': 883179,
 'last_fork_hash': '0000000000000000000162cb30b0517f2de9aac98b3fd7e4b097052ad3097163'}

In [6]:
detail_resp

{'hash': '000000000000000000012389cc51a5057e087221d73fbac77dc53129fcea1edb',
 'height': 891433,
 'chain': 'BTC.main',
 'total': 8126495458,
 'fees': 214215,
 'size': 214020,
 'vsize': 108961,
 'ver': 607666176,
 'time': '2025-04-08T02:05:43Z',
 'received_time': '2025-04-08T02:05:50.518Z',
 'relayed_by': '98.82.224.47:8333',
 'bits': 386027781,
 'nonce': 2893276499,
 'n_tx': 565,
 'prev_block': '00000000000000000001a8839215fe72cfea1cae3d3e1531b3da0ce18c6b759d',
 'mrkl_root': 'b1a7005776145bc4bfa47ad9e6238b9bfbda94154719420f6d700ecd6ff188e9',
 'txids': ['...'],
 'depth': 0,
 'prev_block_url': 'https://api.blockcypher.com/v1/btc/main/blocks/00000000000000000001a8839215fe72cfea1cae3d3e1531b3da0ce18c6b759d',
 'tx_url': 'https://api.blockcypher.com/v1/btc/main/txs/',
 'next_txids': 'https://api.blockcypher.com/v1/btc/main/blocks/000000000000000000012389cc51a5057e087221d73fbac77dc53129fcea1edb?txstart=501&limit=500'}

In [ ]:
# This is specifically for BTC
# check the api url

# Step 1: Get the latest block hash
block_url = "https://api.blockcypher.com/v1/btc/main"
block_resp = requests.get(block_url).json()
block_hash = block_resp['hash']

# Step 2: Get block details using block hash
block_detail_url = f"https://api.blockcypher.com/v1/btc/main/blocks/{block_hash}?txstart=1&limit=500"
detail_resp = requests.get(block_detail_url).json()

# Basic data
block_id = detail_resp.get('hash')
num_transactions = detail_resp.get('n_tx')
timestamp_unix = detail_resp.get('time')  # ISO8601 format

# Convert ISO8601 to GMT datetime
timestamp_gmt = datetime.fromisoformat(timestamp_unix.replace("Z", "+00:00")).strftime("%Y-%m-%d %H:%M:%S GMT")

# Step 3: Get transactions in the block
tx_hashes = detail_resp.get('txids', [])  # List of transaction hashes
tx_values = []
tx_fees = []

for tx_hash in tx_hashes:
    tx_url = f"https://api.blockcypher.com/v1/btc/main/txs/{tx_hash}"
    tx_resp = requests.get(tx_url).json()

    total = tx_resp.get('total', 0) / 1e8  # Convert from satoshis to BTC
    fee = tx_resp.get('fees', 0) / 1e8     # Also convert to BTC

    tx_values.append(total)
    tx_fees.append(fee)

# Step 4: Calculate required values
largest_transaction = max(tx_values) if tx_values else 0
avg_transaction = sum(tx_values) / len(tx_values) if tx_values else 0
avg_fee = sum(tx_fees) / len(tx_fees) if tx_fees else 0

# Final Output
print(f"Block ID: {block_id}")
print(f"Number of Transactions: {num_transactions}")
print(f"Largest Transaction (BTC): {largest_transaction:.8f}")
print(f"Average Transaction (BTC): {avg_transaction:.8f}")
print(f"Average Fee (BTC): {avg_fee:.8f}")
print(f"Time (GMT): {timestamp_gmt}")


Block ID: 000000000000000000012389cc51a5057e087221d73fbac77dc53129fcea1edb
Number of Transactions: 565
Largest Transaction (BTC): 10.19321490
Average Transaction (BTC): 0.06186582
Average Fee (BTC): 0.00000171
Time (GMT): 2025-04-08 02:05:43 GMT
